In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from utils import load_galaxy_data

import app

input_data, labels = load_galaxy_data()
print(input_data.shape) #print shape of features
print(labels.shape) #print shape of labels

#split data into training and validation
x_train, x_valid, y_train, y_valid = train_test_split(input_data, labels, test_size=0.20, stratify=labels, shuffle=True, random_state=222)

#data preprocessing
data_generator = ImageDataGenerator(rescale=1./255) #Creates an ImageDataGenerator and nomalize 

#1) Use data_generator.flow to load the training data with batch size of 5
training_iterator = data_generator.flow(x_train, y_train,batch_size=5)

#2) Use data_generator.flow to load the validation data with batch size of 5
validation_iterator = data_generator.flow(x_valid, y_valid, batch_size=5)

#building the CNN model
model = tf.keras.Sequential(name='Galaxies_Classification') #initilize galaxy sequential model

#add an input layer
model.add(tf.keras.Input(shape=(128,128,3)))

#add Conv2D layer with 8 filters 3x3 size and stride 2
model.add(tf.keras.layers.Conv2D(8, 3, strides=2,activation="relu"))

#Add first max pooling layer 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#add another Conv2D layer with 8 filters 3x3 size and stride 2
model.add(tf.keras.layers.Conv2D(8, 3, strides=2,activation="relu"))

#Add second max pooling layer 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#add flatten layer
model.add(tf.keras.layers.Flatten())

#add hidden dense layer with 16 hidden units
model.add(tf.keras.layers.Dense(16, activation="relu"))

#add a Dense output layer
model.add(tf.keras.layers.Dense(4,activation="softmax"))

#compile the model
# Compile the model with an Adam optimizer, Categorical Cross Entropy Loss, and Accuracy and AUC metrics:

model.compile(
   optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
   loss=tf.keras.losses.CategoricalCrossentropy(),
   metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC()]
)

print(model.summary()) #print the model summary

#Training the model
#4) Use model.fit(...) to train and validate our model for 8 epochs:

model.fit(
       training_iterator,
       steps_per_epoch=len(x_train)/5,
       epochs=8,
       validation_data=validation_iterator,
       validation_steps=len(x_valid)/5)

# predict and visualize
from visualize import visualize_activations
visualize_activations(model, validation_iterator)
